In [1]:
from bs4 import BeautifulSoup
import datetime
from konlpy.tag import Okt
import matplotlib
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import pickle
import re
import requests
import spacy
import textacy.preprocessing as tprep
import time
from tqdm import tqdm
from wordcloud import WordCloud


KeyboardInterrupt



In [ ]:
"""
앞으로 할 것
    1. 토큰화
    2. 언어적 처리
    3. 데이터 수집 성능 향상 -> 데이터 직접 요청하던가 할 것
    4. 시각화, 요약보고

토큰화 방법
    - nltk, spacy, okt 활용
    - 불용어 제거
    - 조사 제거
    - 너무 흔하면서 별 의미 없는 요소 제거해 특성 감소
    - 너무 희소한 요소 제거해 특성 감소
    - ngram 추가해 특성 확장
    - 단어 정규화: 원형 복원
    - 품사 태깅을 통한 특성 선별 (~ 조사 제거)
"""

In [ ]:
def get_max_page(date):

    BASE_URL = 'https://news.naver.com/main/list.naver?mode=LSD&mid=sec&listType=title&'

    last_page = 1000

    while True:
        url = BASE_URL
        url += f'date={date}&'
        url += f'page={last_page}'

        res = requests.get(url)
        bs = BeautifulSoup(res.text)
        time.sleep(1)

        has_next = bs.find('a', class_='next nclicks(fls.page)')
        page_list = bs.find('div', class_='paging')
        
        if not has_next and last_page >= int(page_list.find('strong').get_text()):
            last_page = int(page_list.find('strong').get_text())
            break
        else :
            last_page += 1000


    return last_page

In [ ]:
def crawl(start, end, time_sleep=0.5, page_start=1) :
    BASE_URL = 'https://news.naver.com/main/list.naver?mode=LSD&mid=sec&listType=title&'

    # page = 1
    # date = datetime.datetime.now()
    period = pd.date_range(date_start, date_end)
    
    for ts in period:
        date = str(ts.year) + str(ts.month).zfill(2) + str(ts.day).zfill(2)
        raw_data = {'titles': [], 'dates': []}
        max_page = get_max_page(date)
        print(f"{date} ( MAX PAGE : {max_page} ) : ", end='')
        page = page_start
        pct = 0.1
        
        while True:
            url = BASE_URL
            url += f'date={date}&'
            url += f'page={page}'
    
            res = requests.get(url)
            bs = BeautifulSoup(res.text)
            time.sleep(time_sleep)

            raw_titles = [e.get_text() for e in bs.find_all('a', class_="nclicks(fls.list)")]
            raw_data['titles'].extend(raw_titles)
            raw_data['dates'].extend([date]*len(raw_titles))

            if page / max_page > pct :
                print('*', end='')
                pct += 0.1

            has_next = bs.find('a', class_='next nclicks(fls.page)')
            page_list = bs.find('div', class_='paging')
            
            if not has_next and page == int(page_list.find('strong').get_text()):
                break
            else : 
                page += 1

        print("\tDONE", end=' / ')
        save_raw_data(raw_data, date, 'tst_data')

    return raw_data

In [ ]:
def save_raw_data(raw_data, fname, fpath):
    with open(f'{fpath}/{fname}.pkl', 'wb') as f:
        print('saving... ', end='')
        pickle.dump(raw_data, f)
        print('DONE')

    return True

In [ ]:
def load_raw_data(fname, fpath):
    with open(f'{fpath}/{fname}.pkl', 'rb') as f:
        return pickle.load(f)

In [ ]:
'''
042724 : 20240101 ~ 20240425 크롤링

'''

date_start = '20240101'
date_end = '20240425'

raw_data = crawl(date_start, date_end, time_sleep=0.5, page_start=1)

In [ ]:
load_raw_data('20240127', 'tst_data')

In [ ]:
save_raw_data('20240101_20240426')

In [ ]:
%%time

crawl(20240404, 20240404)

In [ ]:
try:
    with open('raw_data.pkl', 'rb') as f:
        raw_data = pickle.load(f)

except:
    raw_data = crawl()

np.random.choice(raw_data['titles'], 10)

In [ ]:
def get_punct_list(title):
    return re.findall(r'[^ㄱ-ㅎ-가-힣\w\s\(\{\[\)\}\]]', title)

def get_punct_set(titles):
    punct_set = set()
    titles.apply(lambda x: punct_set.update(get_punct_list(x)))
    return punct_set

def get_punct_freq(titles):
    punct_set = get_punct_set(titles)
    punct_freq = {p : 0 for p in punct_set}
    for t in titles:
        for p in get_punct_list(t):
            punct_freq[p] += 1

    return punct_freq

In [ ]:
titles = pd.Series(raw_titles)
punct_set = get_punct_set(titles)
punct_set

In [ ]:
punct_freq = get_punct_freq(titles)
punct_freq

In [ ]:
punct_info = pd.DataFrame([punct_freq.keys(), [ord(p) for p in punct_freq.keys()], punct_freq.values()]).T
punct_info.sort_values(ascending=False, by=2)

In [ ]:
def get_impurity_score(title:str):
    cpy = title[:]
    cpy = re.sub(r'[\(\{\[]+[ㄱ-ㅎ-가-힣\w\s,]+[^ㄱ-ㅎ-가-힣\w\s]*[\]\}\)]+', '.', cpy)
    cpy = re.sub(r'\s', '', cpy)

    n_chars = len(cpy) if len(cpy) != 0 else 1 # (copyright) 같은 제목 때문에 0 발생 -> 1로 처리
    n_puncts = len(get_punct_list(cpy))
    
    return round(n_puncts / n_chars, 3)

In [ ]:
top_10_impurities = titles.apply(get_impurity_score).sort_values(ascending=False).head(10).index
titles[top_10_impurities]

In [ ]:
TRANSLATE_TABLE = { # 치환 후 없앨 것들 목록
    ord(x) : ord(y) 
    for x, y
    in [
        ('［', '['),
        ('％', '%'),
        ('］', ']'),
        ('″', '"'),
        ('”', '"'),
        ('‘', "'"),
        ('∼', '~'),
        ('`', "'"),
        ('’', "'"),
        ('“', '"')
    ]
}

TRANSLATE_TABLE

('↓', 8595) ('&', 38) ('㎝', 13213) ('↑', 8593) ('→', 8594)

In [ ]:
def rep_sokbo_into_ub(title):
    return re.sub(r'[\(\{\[]+[ㄱ-ㅎ-가-힣\w\s,]+[^ㄱ-ㅎ-가-힣\w\s]*[\]\}\)]+', '_', title)

In [ ]:
def normalize_punct(title):

    punct_list = ['\'', '\"', '…', ',', '‥', '!', '@', '#', '&', '/', '+', '=', '~', '?', '>', '_', '㈜', '↓', '↑', '→']
    
    title = rep_sokbo_into_ub(title) # (속보), [단독] 따위의 [000의 건강상식]과 같은 요소들은 .으로 변경
    title = title.translate(TRANSLATE_TABLE)
    
    title = tprep.normalize.quotation_marks(title) # 따옴표 정규화
    
    title = re.sub(r'\.\.(\.)?', '…', title) # 말줄임표 정규화 ('..' , '...' -> '…')
    
    title = tprep.normalize.bullet_points(title)
    title = re.sub(r'·', ' ', title) # 불릿 표현 정규화 + 띄어쓰기로 변형 -> 추후 품사 태깅 등을 통해 낱말 조합 등 진행

    # 필요 없는 문장부호 제거 
    # + '‥' 추가 : 041324 1318
    title = tprep.remove.punctuation(title, only=punct_list)
    
    title = re.sub('\s+', ' ', title) # 위에서 생긴 연속 공백 제거
    title = title.strip() # 양 끝 공백 제거
    
    return title

In [ ]:
clean_titles = titles.apply(normalize_punct)
clean_titles.apply(get_impurity_score).mean()

In [ ]:
okt = Okt()
okt

요약보고

20200101부터 20200108 까지의 기사를 분석한 결과입니다. 해당 기간 동안 (특정 n개 주제)가 새로 주목받기 시작했습니다. 반면 (특정 n개) 에 대한 관심은 줄어드는 추세를 보였습니다. (n개)는 꾸준한 관심을 보였습니다.

- 새로 주목받기 시작한 키워드 : 시작일 경 top n 밖에 있다 새로 들어온 키워드
- 관심 줄어드는 키워드 : 시작일 경 top n에 있다가 밖으로 나간 키워드
- 꾸준한 관심 보이는 키워드 : 기간동안 단어 목록에서 top n 안에 계속 들어온 키워드

In [ ]:
tk = clean_titles.apply(okt.morphs)
tk = tk.apply(lambda x: ' '.join(x))

In [ ]:
tk.sample(10)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
ko_tfidf = TfidfVectorizer()
ko_cntvec = CountVectorizer()

ko_dt = ko_tfidf.fit_transform(tk)
ko_ct = ko_cntvec.fit_transform(tk)

In [ ]:
def get_wc_data_cnt(model, data, is_tfidf=False):
    wc_data = np.c_[model.get_feature_names_out(), data.toarray().sum(axis=0)]
    wc_data = wc_data[wc_data[ : , 1].argsort()]
    wc_data = wc_data[-50 : ]
    wc_data = {item[0]: item[1] for item in wc_data}

    return wc_data

In [ ]:
wc = WordCloud(font_path='C:/Windows/Fonts/malgun.ttf')
cloud = wc.generate_from_frequencies(get_wc_data_cnt(ko_cntvec, ko_ct))

# Display the word cloud using matplotlib
plt.imshow(wc, interpolation='bilinear')
plt.axis("off")

plt.savefig('wc_ko_cntvec.png')

plt.show()

In [ ]:
clean_titles.sample(10).apply(okt.pos)

In [ ]:
okt.phrases(clean_titles[10])

In [ ]:
okt.tagset

In [ ]:
from konlpy import tag

In [ ]:
def custom_tokenize(title, kr_module):

    josa_tag = None

    if isinstance(komoran, tag.Okt): 
        josa_tag = ['Josa']
    elif isinstance(komoran, tag.Komoran):
        josa_tag = ['JKS', 'JKC', 'JKG', 'JKO', 'JKB', 'JKV', 'JKQ', 'JC', 'JX']
    
    title = kr_module.pos(title)
    title = [t for t in title if t[1] not in josa_tag]
    title = [t[0] for t in title]
    title = ' '.join(title)

    return title

In [ ]:
JKS	주격 조사
JKC	보격 조사
JKG	관형격 조사
JKO	목적격 조사
JKB	부사격 조사
JKV	호격 조사
JKQ	인용격 조사
JC	접속 조사
	
JX	보조사
	

In [ ]:
custom_tokenize(clean_titles[0])

In [ ]:
custom_tokenize(clean_titles[2403])

In [ ]:
ctk = clean_titles.apply(custom_tokenize)

custom_ct = ko_cntvec.fit_transform(ctk)

wc = WordCloud(font_path='C:/Windows/Fonts/malgun.ttf')
cloud = wc.generate_from_frequencies(get_wc_data_cnt(ko_cntvec, custom_ct))

# Display the word cloud using matplotlib
plt.imshow(wc, interpolation='bilinear')
plt.axis("off")

plt.savefig('wc_ko_cntvec.png')

plt.show()

In [ ]:
custom_tokenize('간담회 주재하는 김용삼 차관')

In [ ]:
okt.pos('간담회 주재하는 김용삼 차관')

In [ ]:
nlp = spacy.load('ko_core_news_sm')
nlp

In [ ]:
nltk.tokenize.word_tokenize('간담회 주재하는 김용삼 차관')

In [ ]:
from konlpy import tag

In [ ]:
komoran = tag.Komoran()

In [ ]:
tk_komoran = clean_titles.apply(custom_tokenize, kr_module=komoran)
tk_komoran

In [ ]:
nlp_kr = spacy.load('ko_core_news_sm')
nlp_kr

In [ ]:
def display_lemma(title, nlp):
    doc = nlp(title)
    return [(t, t.lemma_) for t in doc]

In [ ]:
tk_spacy = clean_titles.apply(display_lemma, nlp=nlp_kr)
tk_spacy